## Subajust i sobreajust - Parada anticipada
En aquest exercici millorarem l'entrenament d'una xarxa mitjançant la tècnica de la parada anticipada per evitar un sobreajust.

Primer carreguem el [dataset de Spotify](../data/spotify.csv). L'objectiu final serà predir la popularitat d'una cançó basant-te en diverses característiques.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

spotify = pd.read_csv('data/spotify.csv')

X = spotify.copy().dropna()
y = X.pop('track_popularity')
artists = X['track_artist']

features_num = ['danceability', 'energy', 'key', 'loudness', 'mode',
                'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'duration_ms']
features_cat = ['playlist_genre']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

# We'll do a "grouped" split to keep all of an artist's songs in one
# split or the other. This is to help prevent signal leakage.
def group_split(X, y, group, train_size=0.75):
    splitter = GroupShuffleSplit(train_size=train_size)
    train, test = next(splitter.split(X, y, groups=group))
    return (X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test])

X_train, X_valid, y_train, y_valid = group_split(X, y, artists)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
y_train = y_train / 100 # popularity is on a scale 0-100, so this rescales to 0-1.
y_valid = y_valid / 100

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

### Pregunta 1
Defineix una xarxa amb una neurona.

Compila el model amb l'optimitzador Adam i defineix una funció de pèrdua adequada.

Entrena el model amb un tamany de lot de 512 i 50 èpoques.

Representa en un gràfic 

### Pregunta 2
Representa gràficament des de l'època 10.

Creus que la xarxa està sobreentrenada o subajustada?

### Pregunta 3
Afegeix tres capes ocultes amb 128 unitats cadascuna. Torna a entrenar el model i mira les corbes d'aprenentatge.

Ara pareix que estiguen subajustades o sobreentrenades?

### Prregunta 4
Defineix una funció de `callback` per a la parada anticipada:
- que s'haja d'esperar almenys fins a l'època 5
- que tinga almenys una millora de 0,001 en la pèrdua de validació
- que recupere els valors de pesos per a la pèrdua mínima


Reentrena el model amb la parada anticipada definida i torna a mirar les corbes d'entrenament.

### Pregunta 5
Quina és la millora enfront de l'entrenament sense parada anticipada?